In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

# 📊 COMED Dataset - Complete Time Series Analysis
This notebook contains a comprehensive time series analysis of the COMED (Commonwealth Edison) hourly energy consumption dataset.

## STEP 1 — Load and Prepare Data 🕐

In [ ]:
# Load COMED dataset
df = pd.read_csv('./Data/COMED_hourly.csv')

print("Original Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nData Types:")
print(df.dtypes)

# Convert Datetime column to datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set Datetime as index
df.set_index('Datetime', inplace=True)

# Sort by datetime
df.sort_index(inplace=True)

print("\n" + "="*80)
print("✅ Data Loaded and Prepared!")
print("="*80)
print(f"\nDataset Info:")
print(f"Shape: {df.shape}")
print(f"Date Range: {df.index.min()} to {df.index.max()}")
print(f"Total Days: {(df.index.max() - df.index.min()).days} days")
print(f"\nMissing Values: {df.isnull().sum().sum()}")
print(f"\nStatistics:")
print(df.describe())

## STEP 2 — Visualize the Time Series 📈

In [ ]:
# Get energy column name
energy_col = df.columns[0]
print(f"Energy column: {energy_col}")

# Create comprehensive visualizations
fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# 1. Full Time Series
axes[0].plot(df.index, df[energy_col], color='steelblue', linewidth=0.5)
axes[0].set_title('COMED Energy Consumption - Complete Time Series', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Energy (MW)')
axes[0].grid(True, alpha=0.3)

# 2. One Year Pattern
display_year = df.index.year.unique()[0]
df_year = df[df.index.year == display_year]
axes[1].plot(df_year.index, df_year[energy_col], color='darkgreen', linewidth=0.8)
axes[1].set_title(f'{display_year} - Annual Pattern', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Energy (MW)')
axes[1].grid(True, alpha=0.3)

# 3. One Week Pattern
week_start = df.index.min()
week_end = week_start + pd.Timedelta(days=7)
df_week = df[(df.index >= week_start) & (df.index < week_end)]
axes[2].plot(df_week.index, df_week[energy_col], color='crimson', linewidth=1.5, marker='o', markersize=3)
axes[2].set_title(f'Weekly Pattern - First Week (Hourly)', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Date')
axes[2].set_ylabel('Energy (MW)')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## STEP 3 — Feature Engineering 🔥

In [ ]:
# Create time-based features
df_features = df.copy()

# Temporal features
df_features['hour'] = df_features.index.hour
df_features['day'] = df_features.index.day
df_features['month'] = df_features.index.month
df_features['year'] = df_features.index.year
df_features['day_of_week'] = df_features.index.dayofweek
df_features['day_of_year'] = df_features.index.dayofyear
df_features['week_of_year'] = df_features.index.isocalendar().week
df_features['quarter'] = df_features.index.quarter

# Cyclical features
df_features['hour_sin'] = np.sin(2 * np.pi * df_features['hour']/24)
df_features['hour_cos'] = np.cos(2 * np.pi * df_features['hour']/24)
df_features['day_sin'] = np.sin(2 * np.pi * df_features['day_of_week']/7)
df_features['day_cos'] = np.cos(2 * np.pi * df_features['day_of_week']/7)
df_features['month_sin'] = np.sin(2 * np.pi * df_features['month']/12)
df_features['month_cos'] = np.cos(2 * np.pi * df_features['month']/12)

# Lag features
df_features['lag_1h'] = df_features[energy_col].shift(1)
df_features['lag_24h'] = df_features[energy_col].shift(24)
df_features['lag_168h'] = df_features[energy_col].shift(168)

# Rolling statistics
df_features['rolling_mean_24h'] = df_features[energy_col].rolling(window=24).mean()
df_features['rolling_std_24h'] = df_features[energy_col].rolling(window=24).std()
df_features['rolling_min_24h'] = df_features[energy_col].rolling(window=24).min()
df_features['rolling_max_24h'] = df_features[energy_col].rolling(window=24).max()

# Difference features
df_features['diff_1h'] = df_features[energy_col].diff(1)
df_features['diff_24h'] = df_features[energy_col].diff(24)

print("✅ Feature Engineering Complete!")
print(f"Total features created: {len(df_features.columns)}")
print(f"\nFeatures: {df_features.columns.tolist()}")

## STEP 4 — Temporal Pattern Analysis 📊

In [ ]:
# Analyze patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Hourly pattern
hourly_avg = df_features.groupby('hour')[energy_col].mean()
axes[0, 0].bar(hourly_avg.index, hourly_avg.values, color='steelblue', alpha=0.7)
axes[0, 0].set_title('Average Energy by Hour of Day', fontweight='bold')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Average MW')
axes[0, 0].grid(True, alpha=0.3)

# Day of week pattern
dow_avg = df_features.groupby('day_of_week')[energy_col].mean()
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[0, 1].bar(dow_avg.index, dow_avg.values, color='darkgreen', alpha=0.7)
axes[0, 1].set_title('Average Energy by Day of Week', fontweight='bold')
axes[0, 1].set_xlabel('Day of Week')
axes[0, 1].set_ylabel('Average MW')
axes[0, 1].set_xticks(range(7))
axes[0, 1].set_xticklabels(dow_labels)
axes[0, 1].grid(True, alpha=0.3)

# Monthly pattern
monthly_avg = df_features.groupby('month')[energy_col].mean()
axes[1, 0].plot(monthly_avg.index, monthly_avg.values, marker='o', color='darkorange', linewidth=2)
axes[1, 0].set_title('Average Energy by Month', fontweight='bold')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Average MW')
axes[1, 0].set_xticks(range(1, 13))
axes[1, 0].grid(True, alpha=0.3)

# Distribution
axes[1, 1].hist(df[energy_col], bins=50, color='crimson', alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Energy Consumption Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Energy (MW)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 KEY INSIGHTS:")
print(f"Peak Hour: {hourly_avg.idxmax()}:00 ({hourly_avg.max():.2f} MW)")
print(f"Low Hour: {hourly_avg.idxmin()}:00 ({hourly_avg.min():.2f} MW)")
print(f"Peak Day: {dow_labels[dow_avg.idxmax()]} ({dow_avg.max():.2f} MW)")
print(f"Peak Month: {monthly_avg.idxmax()} ({monthly_avg.max():.2f} MW)")

## STEP 5 — Stationarity Check 📉

In [ ]:
# ADF Test
print("="*80)
print("STATIONARITY TESTS")
print("="*80)

ts = df[energy_col].dropna()

# Augmented Dickey-Fuller test
adf_result = adfuller(ts)
print("\n📊 Augmented Dickey-Fuller Test:")
print(f"   ADF Statistic: {adf_result[0]:.6f}")
print(f"   P-value: {adf_result[1]:.6f}")
print(f"   Critical Values:")
for key, value in adf_result[4].items():
    print(f"      {key}: {value:.3f}")

if adf_result[1] <= 0.05:
    print("   ✅ Reject null hypothesis - Series is stationary")
else:
    print("   ⚠️ Fail to reject null hypothesis - Series is non-stationary")

# KPSS Test
kpss_result = kpss(ts, regression='c')
print("\n📊 KPSS Test:")
print(f"   KPSS Statistic: {kpss_result[0]:.6f}")
print(f"   P-value: {kpss_result[1]:.6f}")
print(f"   Critical Values:")
for key, value in kpss_result[3].items():
    print(f"      {key}: {value:.3f}")

if kpss_result[1] >= 0.05:
    print("   ✅ Fail to reject null hypothesis - Series is stationary")
else:
    print("   ⚠️ Reject null hypothesis - Series is non-stationary")

## STEP 6 — Autocorrelation Analysis 🔍

In [ ]:
# Plot ACF and PACF
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

plot_acf(ts, lags=168, ax=axes[0], alpha=0.05)
axes[0].set_title('Autocorrelation Function (ACF)', fontweight='bold', fontsize=12)
axes[0].set_xlabel('Lag (hours)')

plot_pacf(ts, lags=50, ax=axes[1], alpha=0.05)
axes[1].set_title('Partial Autocorrelation Function (PACF)', fontweight='bold', fontsize=12)
axes[1].set_xlabel('Lag (hours)')

plt.tight_layout()
plt.show()

print("✅ ACF/PACF plots show correlation patterns")
print("   • Strong correlations at lag 24 indicate daily seasonality")
print("   • Strong correlations at lag 168 indicate weekly seasonality")

## STEP 7 — Time Series Decomposition 🔥

In [ ]:
# Decompose time series
print("Performing time series decomposition...")

# Use weekly seasonality (168 hours)
period = 168
subset_size = min(2160, len(ts))
ts_subset = ts.iloc[:subset_size]

decomposition = seasonal_decompose(ts_subset, model='additive', period=period, extrapolate_trend='freq')

# Plot decomposition
fig, axes = plt.subplots(4, 1, figsize=(16, 12))

# Original
axes[0].plot(ts_subset.index, ts_subset.values, color='steelblue', linewidth=1)
axes[0].set_title('Original Time Series', fontweight='bold')
axes[0].set_ylabel('Energy (MW)')
axes[0].grid(True, alpha=0.3)

# Trend
axes[1].plot(decomposition.trend.index, decomposition.trend.values, color='darkgreen', linewidth=2)
axes[1].set_title('Trend Component', fontweight='bold')
axes[1].set_ylabel('Trend (MW)')
axes[1].grid(True, alpha=0.3)

# Seasonal
axes[2].plot(decomposition.seasonal.index, decomposition.seasonal.values, color='darkorange', linewidth=1)
axes[2].set_title('Seasonal Component', fontweight='bold')
axes[2].set_ylabel('Seasonal (MW)')
axes[2].grid(True, alpha=0.3)

# Residual
axes[3].plot(decomposition.resid.index, decomposition.resid.values, color='crimson', linewidth=0.8)
axes[3].set_title('Residual Component', fontweight='bold')
axes[3].set_xlabel('Date')
axes[3].set_ylabel('Residual (MW)')
axes[3].axhline(y=0, color='black', linestyle='--', linewidth=1, alpha=0.5)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Decomposition complete!")
print(f"   • Trend shows long-term changes")
print(f"   • Seasonal shows repeating patterns")
print(f"   • Residual shows random fluctuations")

## Summary 📋

This notebook completed:
1. ✅ Data loading and preparation
2. ✅ Time series visualization
3. ✅ Feature engineering
4. ✅ Temporal pattern analysis
5. ✅ Stationarity testing
6. ✅ Autocorrelation analysis
7. ✅ Time series decomposition

**Next Steps:**
- Model selection (ARIMA, SARIMA, Prophet, LSTM)
- Train-test split
- Model training and evaluation
- Forecasting